In [1]:
### Own - Conda venv --- dc_info_venv
# Source --- https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564
### main Source --- https://www.tensorflow.org/guide/

# 
import tensorflow as tf
#from tf.keras import layers ### Fails - We have TF version == 1.5.0 

import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline
np.random.seed(1)
#
print(tf.VERSION)
print(tf.keras.__version__)
import keras
print('Keras: {}'.format(keras.__version__))

1.5.0
2.1.2-tf
Keras: 2.2.4


Using TensorFlow backend.


In [2]:
# Source --- https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564

"""
A TFRecord file stores your data as a sequence of binary strings.
This means you need to specify the structure of your data before you write it to the file.
Tensorflow provides two components for this purpose: 

tf.train.Example and 
tf.train.SequenceExample. 

You have to store each sample of your data in one of these structures, 
then ----serialize-------- it and use a tf.python_io.TFRecordWriter to write it to disk.
"""

## DHANKAR --- FATT --- Some other sources mentinn getting IMAGES in as NUMPY ARRAYS ?
## SOURCE ---- https://www.tensorflow.org/api_docs/python/tf/data


"""
The tf.data API enables you to build complex input pipelines from simple, reusable pieces.
For example, the pipeline for an image model might aggregate data from files in a ---- distributed file system,
apply random perturbations to each image, and ------- merge randomly selected images ---- into a batch for training.

The pipeline for a text model might involve extracting symbols from raw text data, converting
them to embedding identifiers with a ----lookup table-----, and -----batching together sequences----
of different lengths. 

The tf.data API makes it easy to deal with large amounts of data,
different data formats, and complicated transformations.
"""

### tensor_1 == image_data
### tensor_2 == image_label
"""
A tf.data.Dataset represents a sequence of elements, in which each element contains one or more ---Tensor-- objects.
For example, in an--- image pipeline, an element might be a ----single training example---, with a pair of tensors
representing the image data and a label.
"""

### Dataset.from_tensor_slices()
### Dataset.batch()

"""

    Creating a source (e.g. Dataset.from_tensor_slices()) constructs a dataset from one or more tf.Tensor objects.

    Applying a transformation (e.g. Dataset.batch()) constructs a dataset from one or more tf.data.Dataset objects.

"""


### tf.data.Iterator

"""
A tf.data.Iterator provides the main way to extract elements from a dataset. 
The operation returned by Iterator.get_next() yields the next element of a Dataset when executed,
and typically acts as the interface between input pipeline code and your model.

The simplest iterator is a "one-shot iterator", which is associated with a particular Dataset and 
iterates through it once.

For more sophisticated uses, the Iterator.initializer operation enables you to reinitialize 
and parameterize an iterator with different datasets, 
so that you can, for example, 
iterate over training and validation data multiple times in the same program.
"""

### Dataset structure
# --- dataset >> elements >> tf.Tensor -- components >> tf.TensorShape

"""
Dataset structure

A dataset comprises ---elements--- that each have the same structure. 
An element contains one or more ----tf.Tensor objects---, called ----components---.
----- Each component has a tf.DType representing the type of elements in the tensor
----- and a tf.TensorShape representing the (possibly partially specified) static shape of each element. 
"""

### PROPERTIES ===>>  Dataset.output_types and Dataset.output_shapes
"""
The Dataset.output_types and Dataset.output_shapes properties 

----allow you to inspect the inferred types 
----and shapes of each component of a dataset element. 

The nested structure of these properties map to the structure of an element, 
--- which may be a single tensor, 
--- a tuple of tensors, 
--- or a nested tuple of tensors.
"""

### 
"""

"""

In [8]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random_uniform([4, 1000]))
print(dataset1.output_types)  # ==> "tf.float32"
print(dataset1.output_shapes)  # ==> "(10,)"
#
print(dataset1)

<dtype: 'float32'>
(1000,)
<TensorSliceDataset shapes: (1000,), types: tf.float32>


In [6]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random_uniform([4]),
    tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)))
print(dataset2.output_types)  # ==> "(tf.float32, tf.int32)"
print(dataset2.output_shapes)  # ==> "((), (100,))"
#
print(dataset2)


(tf.float32, tf.int32)
(TensorShape([]), TensorShape([Dimension(100)]))
<TensorSliceDataset shapes: ((), (100,)), types: (tf.float32, tf.int32)>


In [5]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))
print(dataset3.output_types)  # ==> (tf.float32, (tf.float32, tf.int32))
print(dataset3.output_shapes)  # ==> "(10, ((), (100,)))"


(tf.float32, (tf.float32, tf.int32))
(TensorShape([Dimension(10)]), (TensorShape([]), TensorShape([Dimension(100)])))


In [ ]:
"""
It is often convenient to give names to each component of an element, 
for example if they represent different features of a training example. 

In addition to tuples, you can use collections.namedtuple or a dictionary mapping strings to tensors 
to represent a single element of a Dataset.
"""



In [9]:
### Official 
dataset = tf.data.Dataset.from_tensor_slices(
   {"a": tf.random_uniform([4]),
    "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)})
print(dataset.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset.output_shapes)  # ==> "{'a': (), 'b': (100,)}"


{'b': tf.int32, 'a': tf.float32}
{'b': TensorShape([Dimension(100)]), 'a': TensorShape([])}


In [12]:
## DHANKAR ---

dataset_11 = tf.data.Dataset.from_tensor_slices(
   {
    "a": tf.random_uniform([4, 500], maxval=1000, dtype=tf.int32),
    "b": tf.random_uniform([4, 100], maxval=100, dtype=tf.int32)
    }
      )

print(dataset_11.output_types)  # ==> "{'a': tf.float32, 'b': tf.int32}"
print(dataset_11.output_shapes)  # ==> "{'a': (), 'b': (100,)}"


{'b': tf.int32, 'a': tf.int32}
{'b': TensorShape([Dimension(100)]), 'a': TensorShape([Dimension(500)])}


In [13]:
### CSV Uploads --ERROR --- FATT 
# latest version of TF == has the CSV Func 
## Documentation for version --- 1.12 
# https://www.tensorflow.org/api_docs/python/tf/contrib/data/CsvDataset

#v-1.6.0 --- Has Experimental --
##- tensorflow/tensorflow/python/data/experimental/benchmarks/csv_dataset_benchmark.py


# Right now using - v-1.5.0 --- which does not . 
# /a6_18/OwnFork_TensorFlow/tensorflow/tensorflow/contrib/data/python/ops/readers.py


# Creates a dataset that reads all of the records from two CSV files, each with
# eight float columns
filenames = ["/media/dhankar/Dhankar_1/a6_18/Tensors_et_al/date_fmts.csv"]

record_defaults = [tf.float32] * 8   # Eight required float columns
dataset = tf.contrib.data.CsvDataset(filenames, record_defaults)

AttributeError: module 'tensorflow.contrib.data' has no attribute 'CsvDataset'